In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio

In [ ]:
BATCH_SIZE = 100
SAVE_RESULTS = True
id_name=ID_DS_LIST[0] # selects the ID dataset.
id_model=ID_MODEL_LIST[0]  # select the deep model used for training ID dataset.
print(id_name,id_model)

In [ ]:
ood_appr_name = OOD_APPROACHES[0] # selects the OOD detection approach.
print("ID dataset:", id_name)
print("Model: ", id_model)
print("OOD approach name:",ood_appr_name )

In [ ]:
(org_traing_data, org_training_labels),(id_eva_data,y_test)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

In [ ]:
x_test = id_eva_data_processed
# y_test = to_categorical(y_test, 10)
org_model = load_custom_model_for_ds(id_name, id_model)
# org_model.summary()
# sc = org_model.evaluate(x_test,y_test)
# print(sc)

In [ ]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [ ]:
def prepare_models(in_org_model, in_ood_appr_name):
    f_model=None
    f_grad_graph = None
    f_osvm = None
    f_ss = None
    if in_ood_appr_name == "softmax":
        f_model = in_org_model
        
    elif in_ood_appr_name=="odin_wo_p":
        scaled_model = apply_temp_scale_to_model(in_org_model, REM_TOP_LAYER)
        f_model = (scaled_model,)
        
    elif in_ood_appr_name=="odin_w_p":
        scaled_model = apply_temp_scale_to_model(in_org_model, REM_TOP_LAYER)
        scaled_log_model = apply_log_temp_scale_to_model(in_org_model, REM_TOP_LAYER, t_value=1000)
        grads = calculate_input_gradients(scaled_log_model)
        f_model = (scaled_model, grads, scaled_log_model )
        
    elif in_ood_appr_name=="mah_dist_logits" or in_ood_appr_name=="mah_dist_logits_w_p":
        tmp_l=in_org_model.layers[REM_TOP_LAYER]
        feature_model = Model(inputs=in_org_model.input, outputs=tmp_l.output)
        sample_class_mean, sigma_inverse =  cal_mah_dist_parameters(feature_model, NUM_CLASS,
                                                                    org_traing_data_processed, org_training_labels)
        m_model = get_inference_mah_model(in_org_model, feature_model, sample_class_mean, sigma_inverse, NUM_CLASS)
        if in_ood_appr_name=="mah_dist_logits": 
            f_model=m_model
        elif in_ood_appr_name=="mah_dist_logits_w_p":
            grads = calculate_input_gradients(m_model)
            f_model=(m_model, grads)
            
    elif in_ood_appr_name=="ours" :
        tmp_l=in_org_model.layers[layer_inx]
        feature_model = Model(inputs=in_org_model.input, outputs=tmp_l.output)
        features = extract_layer_features(feature_model,org_traing_data_processed,layer_inx,BATCH_SIZE,\
                                          shorten_model=True )
        osvm, ss = build_one_class_svm(features)
        f_model = (feature_model, osvm, ss)
       
    return f_model

In [ ]:
def detect_OOD_inputs(in_org_model, in_model, in_ood_appr_name,in_mix_id_odd, in_extra_data):
    if in_ood_appr_name=="softmax":
        scores = detect_ood_softmax(in_org_model, in_model, in_mix_id_odd)
    elif in_ood_appr_name=="odin_wo_p":
        scores = detect_ood_odin(in_org_model, in_model, in_mix_id_odd, in_extra_data, input_preprocessing=False)
    elif in_ood_appr_name=="odin_w_p":
        scores = detect_ood_odin(in_org_model, in_model, in_mix_id_odd, in_extra_data, input_preprocessing=True)
    elif in_ood_appr_name=="mah_dist_logits":
        scores = detect_mah_dist(in_org_model, in_model, in_mix_id_odd,  in_extra_data, NUM_CLASS,                                                                                            input_preprocessing=False)
    elif in_ood_appr_name=="mah_dist_logits_w_p":
        scores = detect_mah_dist(in_org_model, in_model, in_mix_id_odd,  in_extra_data, NUM_CLASS,                                                                                             input_preprocessing=True)
    elif in_ood_appr_name=="ours":
        scores = detect_ood_svm(in_org_model, in_model, in_mix_id_odd,  in_extra_data, input_preprocessing=False)
    return scores

In [ ]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.vstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [ ]:
ood_appr_name = OOD_APPROACHES[0]
detect_model= prepare_models(org_model,ood_appr_name)

In [ ]:
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    ood_eva_data_processed    = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    mixed_data, mixed_labels = combine_inliners_outliers(id_eva_data_processed,ood_eva_data_processed)
    if ood_appr_name in ("ours_w_p", "odin_w_p", "mah_dist_logits_w_p"):
        extra_data = (get_pert_magnitude(ood_appr_name, org_model, detect_model, id_eva_data_processed,\
                                                                            ood_eva_data_processed, NUM_CLASS ) ,)
    else:
        extra_data=None

    scores = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)
    fpr, tpr = nums(scores, mixed_labels)
    ROC = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    print('The results of %s:\n' %(ood_ds_name))
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

In [ ]:
ood_ds_name = 'foolingimages'
ood_eva_data_processed = np.load('ood_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))
mixed_data, mixed_labels = combine_inliners_outliers(id_eva_data_processed,ood_eva_data_processed)
if ood_appr_name in ("ours_w_p", "odin_w_p", "mah_dist_logits_w_p"):
    extra_data = (get_pert_magnitude(ood_appr_name, org_model, detect_model, id_eva_data_processed,\
                                                                        ood_eva_data_processed, NUM_CLASS ) ,)
else:
    extra_data=None

scores = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)
fpr, tpr = nums(scores, mixed_labels)
ROC = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]
FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
print('The results of %s:\n' %(ood_ds_name))
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")

In [ ]:
def prepare_models_ours(in_org_model, in_ood_appr_name,traindatas):
    f_model=None
    f_grad_graph = None
    f_osvm = None
    f_ss = None
    tmp_l=in_org_model.layers[layer_inx]
    feature_model = Model(inputs=in_org_model.input, outputs=tmp_l.output)
    features = extract_layer_features(feature_model,traindatas,layer_inx,BATCH_SIZE,\
                                      shorten_model=True )
    osvm, ss = build_one_class_svm(features)
    f_model = (feature_model, osvm, ss)        
    return f_model

In [ ]:
def normalize(x):
	return (x - np.min(x))/(np.max(x) - np.min(x))

In [ ]:
trainimg = np.load('classdatas_id/trainimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)
testimg = np.load('classdatas_id/testimgs_%s_%s.npy'%(id_model,id_name),allow_pickle=True)

In [ ]:
for i, ds_name in enumerate(OOD_DS_LIST):
    ood = id_name+"_"+id_model+"_"+ ds_name
    images = np.load('classdatas_ood/'+ood+'.npy',allow_pickle=True)
    lists = []
    for i in range(10):
        if (len(images[i]) != 0):
            lists.append(i)
    fpr = dict()
    tpr = dict()
    FP = dict()
    TN = dict()
    TP = dict()
    FN = dict()
    temp = []
    la = []
    ROC = 0
    numclass = len(images)
    for i in lists:
        if id_model=='LeNet':
            layer_inx = 0
        elif id_model=='VGG':
            layer_inx = 6
        elif id_model=='ResNet':
            if id_name=='CIFAR10':
                layer_inx = 20
            elif id_name=='SVHN':
                layer_inx = 24
        train = np.asarray(trainimg[i])
        test = np.asarray(testimg[i])
        ood = np.asarray(images[i])

        detect_model = prepare_models_ours(org_model,ood_appr_name, train)
        mixed_data, mixed_labels = combine_inliners_outliers(test, ood)

        scores1 = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)
        scores1 = normalize(scores1)        
        layer_inx = -2
        detect_model = prepare_models_ours(org_model,ood_appr_name, train)
        mixed_data, mixed_labels = combine_inliners_outliers(test, ood)    
        scores2 = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)
        scores2 = normalize(scores2)
        scores = 0.5*scores1 + 0.5*scores2

        fpr[i], tpr[i] = nums(scores, mixed_labels)
        ROC = auc(fpr[i], tpr[i])
        lens = test.shape[0]
        FP[i],TN[i],TP[i],FN[i] = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    #     print("%d th AUROC: %f"%(i,ROC))   
        temp.append(ROC)
        la.append(mixed_labels)
    for i in range(len(lists)):
        ROC += float(temp[i])       
    
    numbers_test = 0
    number_ood = 0
    for i in lists:
        numbers_test+= len(testimg[i])
        number_ood += len(images[i])
    numbers_all =  numbers_test +  number_ood

    ROC_AVE = 0
    for i in range(len(lists)):
        ROC_AVE += float(len(la[i]/numbers_all)*temp[i])
    ROC_AVE = ROC_AVE/numbers_all

    NFP = 0
    NTN = 0
    NTP = 0
    NFN = 0
    for i in lists:
        NFP += FP[i]
    for i in lists:
        NTN += TN[i]
    for i in lists:
        NTP += TP[i]
    for i in lists:
        NFN += FN[i]
    print('The results of %s:\n'%(ds_name))
    print('tnr_at_95_tpr:', float(NTN) / float(NFP + NTN+ 1e-7) *100)
    print('detection_acc:',(float(NTP) / float(NTP + NFN + 1e-7)+ float(NTN) / float(NFP + NTN + 1e-7))/2*100)
    print("Average AUROC: ",ROC_AVE*100)
    print("\n")

In [ ]:
ds_name = 'fooling images'
ood = id_name+"_"+id_model+"_"+ 'fooling_images'
images = np.load('classdatas_ood/'+ood+'.npy',allow_pickle=True)
lists = []
for i in range(10):
    if (len(images[i]) != 0):
        lists.append(i)
fpr = dict()
tpr = dict()
roc_auc = dict()
FP = dict()
TN = dict()
TP = dict()
FN = dict()
temp = []
la = []
ROC = 0
numclass = len(images)
for i in lists:
    if id_model=='LeNet':
        layer_inx = 0
    elif id_model=='VGG':
        layer_inx = 6
    elif id_model=='ResNet':
        if id_name=='CIFAR10':
            layer_inx = 20
        elif id_name=='SVHN':
            layer_inx = 24
    train = np.asarray(trainimg[i])
    test = np.asarray(testimg[i])
    ood = np.asarray(images[i])

    detect_model = prepare_models_ours(org_model,ood_appr_name, train)
    mixed_data, mixed_labels = combine_inliners_outliers(test, ood)
    scores1 = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)

    layer_inx = -2
    detect_model = prepare_models_ours(org_model,ood_appr_name, train)
    mixed_data, mixed_labels = combine_inliners_outliers(test, ood)    
    scores2 = detect_OOD_inputs(org_model, detect_model, ood_appr_name, mixed_data, extra_data)

    scores = 0.5*scores1 + 0.5*scores2

    lens = test.shape[0]
    fpr[i], tpr[i] = nums(scores, mixed_labels)
    ROC = auc(fpr[i], tpr[i])
    FP[i],TN[i],TP[i],FN[i] = ErrorRateAt95Recall1(lens,scores, mixed_labels)
#     print("%d th AUROC: %f"%(i,ROC))   
    temp.append(ROC)
    la.append(mixed_labels)
      
numbers_test = 0
number_ood = 0
for i in lists:
    numbers_test+= len(testimg[i])
    number_ood += len(images[i])
numbers_all =  numbers_test +  number_ood
# print('numbers_all is:',numbers_all)

ROC_AVE = 0
for i in range(len(lists)):
    ROC_AVE += float(len(la[i]/numbers_all)*temp[i])
ROC_AVE = ROC_AVE/numbers_all

NFP = 0
NTN = 0
NTP = 0
NFN = 0
for i in lists:
    NFP += FP[i]
for i in lists:
    NTN += TN[i]
for i in lists:
    NTP += TP[i]
for i in lists:
    NFN += FN[i]
print('The results of %s:\n'%(ds_name))
print('tnr_at_95_tpr:', float(NTN) / float(NFP + NTN+ 1e-7) *100)
print('detection_acc:',(float(NTP) / float(NTP + NFN + 1e-7)+ float(NTN) / float(NFP + NTN + 1e-7))/2*100)
print("Average AUROC: ",ROC_AVE*100)
print("\n")
